# Plan
- [x] Load all reviews into a data structure in memory
- [x] Apply tokenization to each review (base step)
- [ ] Choose and apply three pathways through the feature generation process
    - _Remember that you will need to justify your choices with reference to the accuracy that is achieved with each feature set_

A simple idea for a datastructure that holds the dataset in memory is:
```Python
dataset = [
    {
        "id": 1,
        "rating": 7,
        "polarity": 1,
        "contents:": "Lorem Ipsum"
    },
    ...
]
```

Later, this should be a `pandas` dataframe.

## Imports & Constants

In [10]:
import os
import copy
import math
import string
from typing import Self
from collections import Counter
from functools import cached_property
from joblib import Parallel, delayed

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams, everygrams

POSITIVE_REVIEWS_DIR = "./data/pos/"
NEGATIVE_REVIEWS_DIR = "./data/neg/"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wij21\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wij21\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wij21\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\wij21\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Dataset Types

In [2]:
class TextualDataPoint:

    def __init__(self, file_path: str):
        self.file_path = file_path
        self.contents = self._extract_contents()

    @cached_property
    def basename(self) -> str:
        return os.path.basename(self.file_path)

    @cached_property
    def filename(self) -> str:
        return os.path.splitext(self.basename)[0]
    
    def _extract_contents(self) -> str:
        with open(self.file_path, "r") as file:
            return file.read()
        
    def as_dict(self) -> dict:
        return {
            'contents': self.contents,
        }


class Review(TextualDataPoint):

    def __init__(self, file_path: str):
        super().__init__(file_path)
        self.id = self._parse_id()
        self.rating = self._parse_rating()
        self.polarity = self._determine_polarity()

    def _parse_id(self) -> int:
        return self.filename.split("_")[0]

    def _parse_rating(self) -> int:
        return int(self.filename.split("_")[1])
    
    def _determine_polarity(self) -> int:
        if self.rating is None:
            self.rating = self._parse_rating()

        if self.rating <= 4:
            return 0
        elif self.rating >= 7:
            return 1
        else:
            raise ValueError(f"unexpected rating: {self.rating}")

    def as_dict(self) -> dict:
        return {
            'id': self.id,
            'rating': self.rating,
            'polarity': self.polarity,
        } | super().as_dict()

In [3]:
class IterableSet:

    datapoint_class = TextualDataPoint

    def __init__(self, datapoints: list[datapoint_class]):
        self.datapoints = datapoints

        # To keep track of the current iteration position.
        self.index = 0

    def first(self) -> datapoint_class:
        return self.datapoints[0]

    def last(self) -> datapoint_class:
        return self.datapoints[-1]
    
    def __len__(self) -> int:
        return len(self.datapoints)
    
    def __iter__(self) -> Self:
        # Reset the index whenever starting a new iteration.
        self.index = 0
        return self
        
    def __next__(self) -> datapoint_class:
        # Make sure there are more datapoints to yield.
        if self.index < len(self.datapoints):
            result = self.datapoints[self.index]
            self.index += 1
            return result
        else:
            # No more datapoints -> raise StopIteration exception.
            raise StopIteration

    def as_lower_representation(self) -> list[dict]:
        return [
            datapoint.as_dict()
            for datapoint in self.datapoints
        ]


class DataSet(IterableSet):

    def __init__(self, dirs: list[str]):
        super().__init__(None)
        self.dirs = dirs
        
    def load(self) -> Self:
        self.datapoints = [
            self.datapoint_class(directory + file)
            for directory in self.dirs
            for file in os.listdir(directory)
        ]

        return self
    
    def as_lower_representation(self) -> list[dict]:
        # Ensure the dataset has been loaded.
        if self.datapoints is None:
            self.load()

        return super().as_lower_representation()

    def __iter__(self) -> Self:
        # Ensure the dataset has been loaded.
        if self.datapoints is None:
            self.load()

        return super().__iter__()
    

class ReviewDataSet(DataSet):

    datapoint_class = Review

## Preprocessing

In [4]:
class Preprocessor:

    def __init__(self, dataset: DataSet):
        # We don't want to modify the original dataset.
        self.dataset = copy.deepcopy(dataset)
        # Tokenization is the first preprocessing step of most NLP applications.
        self.tokenize()

    def tokenize(self) -> Self:
        for datapoint in self.dataset:
            if isinstance(datapoint.contents, list):
                # This datapoint has already been tokenized.
                continue

            datapoint.contents = nltk.word_tokenize(datapoint.contents)
        
        return self

## Feature Set Generation

[Stemming (NTLK)](https://www.nltk.org/howto/stem.html)  
[A comparison of Stemming Algorithms & Lemmatization Algorithms](https://stackoverflow.com/questions/24647400/what-is-the-best-stemming-method-in-python)
- `PorterStemmer` is apparently one of the most aggresive `nltk` stemmers
    - It appears the choice of stemmer has a significant impact on performance
- `SnowballStemmer` appears to be a lighter middle-ground
- Lemmatizers are usually "lighter" than stemmers, but they cannot handle unknown words

In [5]:
class FeatureSet(IterableSet):

    def __init__(self, dataset: DataSet):
        super().__init__(dataset.datapoints)

    def compare_with(self, other_set: Self):
        set1_dp1 = self.first().contents
        set2_dp1 = other_set.first().contents
        max_length_set1 = len(max(set1_dp1, key=len))

        print("Comparing the first datapoint in feature sets A and B respectively:")
        for token1, token2 in zip(set1_dp1, set2_dp1):
            empty_space = " " * (max_length_set1 - len(token1))
            print(f"Set A: {token1} {empty_space}| Set B: {token2}")

In [6]:
class FeatureSetGenerator(Preprocessor):

    def create_n_grams(self, n: int) -> FeatureSet:
        for datapoint in self.dataset:
            datapoint.contents = list(everygrams(datapoint.contents, max_len=n))

        return FeatureSet(self.dataset)
    
    def to_lowercase(self) -> Self:
        for datapoint in self.dataset:
            datapoint.contents = [token.lower() for token in datapoint.contents]

        return self
    
    def remove_stopwords(self) -> Self:
        distinct_stopwords = set(stopwords.words('english'))

        for datapoint in self.dataset:
            datapoint.contents = [token for token in datapoint.contents if token not in distinct_stopwords]

        return self
    
    def remove_punctuation(self) -> Self: 
        for datapoint in self.dataset:
            datapoint.contents = [token for token in datapoint.contents if token not in string.punctuation]

        return self

    def lemmatize(self) -> Self:
        lmtzr = WordNetLemmatizer()

        for datapoint in self.dataset:
            datapoint.contents = [lmtzr.lemmatize(token) for token in datapoint.contents]

        return self

    def stem(self) -> Self:
        # Making the assumption that all datapoints are in English.
        stmr = SnowballStemmer("english")

        for datapoint in self.dataset:
            datapoint.contents = [stmr.stem(token) for token in datapoint.contents]

        return self

## Feature Set Normalisation

In [7]:
class FeatureSetNormalizer:

    def __init__(self, feature_set: FeatureSet):
        # We don't want to modify the original feature set.
        self.feature_set = copy.deepcopy(feature_set)

        self.normalized = False
        self.shared_vocabulary = self._collect_shared_vocabulary()
        self.idfs = self._calculate_idfs()

    @cached_property
    def num_datapoints(self) -> int:
        return len(self.feature_set)

    def perform_tf(self) -> FeatureSet:
        self._count_term_frequencies()

        for datapoint in self.feature_set:
            datapoint.normalized_contents = [
                datapoint.term_frequencies.get(token, 0)
                for token in self.shared_vocabulary
            ]

        return self.feature_set

    def perform_tf_idf(self) -> FeatureSet:
        self._count_term_frequencies()

        for datapoint in self.feature_set:
            datapoint.normalized_contents = [
                (datapoint.term_frequencies.get(token, 0) * self.idfs.get(token))
                for token in self.shared_vocabulary
            ]

        return self.feature_set

    def peform_ppmi(self) -> FeatureSet:
        raise NotImplementedError

    def _collect_shared_vocabulary(self) -> set:
        return sorted({
            token
            for datapoint in self.feature_set
            for token in datapoint.contents
        })
    
    def _calculate_idfs(self) -> dict:
        # We calculate the document frequencies by creating a unique set of tokens for
        # each datapoint (i.e., for each document in the set, counting each token once
        # per document regardless of its frequency within the document itself). The Counter
        # then aggregates these sets across all datapoints, counting the number of documents
        # in which each distinct token appears. This gives us the document frequency for
        # each term in the `shared_vocabulary`.
        document_frequencies = Counter(
            token
            for datapoint in self.feature_set
            for token in set(datapoint.contents)
        )

        return {
            token: math.log(self.num_datapoints / (doc_frequency + 1))
            for token, doc_frequency in document_frequencies.items()
        }
    
    def _count_term_frequencies(self) -> None:
        for datapoint in self.feature_set:
            datapoint.term_frequencies = Counter(datapoint.contents)


In [8]:
dataset = ReviewDataSet([POSITIVE_REVIEWS_DIR, NEGATIVE_REVIEWS_DIR]).load()

In [148]:
feature_set_b = FeatureSetGenerator(dataset).remove_punctuation().lemmatize().create_n_grams(2)
feature_set_c = FeatureSetGenerator(dataset).remove_punctuation().stem().create_n_grams(2)

In [149]:
feature_set_b.compare_with(feature_set_c)

Comparing the first datapoint in feature sets A and B respectively:
Set A: ('Homelessness',)  | Set B: ('homeless',)
Set A: ('Homelessness', 'or') | Set B: ('homeless', 'or')
Set A: ('or',)  | Set B: ('or',)
Set A: ('or', 'Houselessness') | Set B: ('or', 'houseless')
Set A: ('Houselessness',)  | Set B: ('houseless',)
Set A: ('Houselessness', 'a') | Set B: ('houseless', 'as')
Set A: ('a',)  | Set B: ('as',)
Set A: ('a', 'George') | Set B: ('as', 'georg')
Set A: ('George',)  | Set B: ('georg',)
Set A: ('George', 'Carlin') | Set B: ('georg', 'carlin')
Set A: ('Carlin',)  | Set B: ('carlin',)
Set A: ('Carlin', 'stated') | Set B: ('carlin', 'state')
Set A: ('stated',)  | Set B: ('state',)
Set A: ('stated', 'ha') | Set B: ('state', 'has')
Set A: ('ha',)  | Set B: ('has',)
Set A: ('ha', 'been') | Set B: ('has', 'been')
Set A: ('been',)  | Set B: ('been',)
Set A: ('been', 'an') | Set B: ('been', 'an')
Set A: ('an',)  | Set B: ('an',)
Set A: ('an', 'issue') | Set B: ('an', 'issu')
Set A: ('issu

In [150]:
normalizer = FeatureSetNormalizer(feature_set_c)

In [151]:
normalized_feature_set = normalizer.perform_tf_idf()

In [145]:
len(normalized_feature_set.first().normalized_contents)

32207

In [146]:
normalized_feature_set.first().normalized_contents

[0.0,
 0.7476033663560041,
 0.0,
 0.0,
 0.0,
 0.0,
 2.3496011582187513,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

## Splitting of sets